In [1]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler


In [2]:
df = pd.read_csv('Cleaned_Data.csv')
test = pd.read_csv('Cleaned_Data.csv').tail(3778)

In [3]:
df

,Name,Certificate,Poster,Genre,Director,Writer,Stars,Storyline,Rating,Rating Count
0,The Dark Knight,PG-13,https://m.media-amazon.com/images/M/MV5BMTMxNT...,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...,9.0,2800000.0
1,The Lord of the Rings: The Return of the King,PG-13,https://m.media-amazon.com/images/M/MV5BNzA5ZD...,"Action, Adventure, Drama",Peter Jackson,"J.R.R. Tolkien, Fran Walsh, Philippa Boyens","Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",Gandalf and Aragorn lead the World of Men agai...,9.0,1900000.0
2,Dr. Babasaheb Ambedkar,Unrated,https://m.media-amazon.com/images/M/MV5BNGNjZD...,"Biography, History",Jabbar Patel,"Daya Pawar, Arun Sadhu, Sooni Taraporevala","Mammootty, Sonali Kulkarni, Mohan Gokhale, Mri...","Biopic of B. R. Ambedkar, known mainly for his...",8.9,2700.0
3,The Lord of the Rings: The Fellowship of the Ring,PG-13,https://m.media-amazon.com/images/M/MV5BN2EyZj...,"Action, Adventure, Drama",Peter Jackson,"J.R.R. Tolkien, Fran Walsh, Philippa Boyens","Elijah Wood, Ian McKellen, Orlando Bloom, Sean...",A meek Hobbit from the Shire and eight compani...,8.8,2000000.0
4,The Lord of the Rings: The Two Towers,PG-13,https://m.media-amazon.com/images/M/MV5BZGMxZT...,"Action, Adventure, Drama",Peter Jackson,"J.R.R. Tolkien, Fran Walsh, Philippa Boyens","Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",While Frodo and Sam edge closer to Mordor with...,8.8,1700000.0
...,...,...,...,...,...,...,...,...,...,...
23773,The Lives of Mount Druitt Youth,Unrated,https://m.media-amazon.com/images/M/MV5BMTI4MT...,"Documentary, Drama",Saad Adam,NaN,"Drew Pearson, Saad Adam, Evan Adam",Come with Saad as he visits Mount Druitt &amp;...,4.8,1531.0
23774,Hich,Unrated,https://m.media-amazon.com/images/M/MV5BYTY0Y2...,"Comedy, Drama",Abdolreza Kahani,"Abdolreza Kahani, Hossein Mahkam","Ahmad Mehranfar, Mehran Ahmadi, Nayereh Farahani","A man, who has some kind of a disease and eats...",6.8,1128.0
23775,Kummeli Alivuokralainen,Unrated,https://m.media-amazon.com/images/M/MV5BODI2MD...,Comedy,Matti Grönberg,"Timo Kahilainen, Heikki Vihinen, Anttu Harlin","Mikko Kivinen, Heikki Silvennoinen, Timo Kahil...","A farce about blackmailing, a kidney transplan...",4.3,1398.0
23776,Någon Annanstans,Unrated,https://m.media-amazon.com/images/M/MV5BMjA0Nz...,"Drama, Romance",Hiwa Abbasi,Hiwa Abbasi,"Ruth Anianson, Arefeh Behbakht, Lina Burström","Said, the main character in the film Another P...",4.4,3333.0


In [4]:
df = df.dropna(subset=['Writer'])
test = test.dropna(subset=['Writer'])

In [5]:
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')

stopwords_list
stopwords_list.append(" ")
stopwords_list.append("")
stopwords_list.extend([chr(i) for i in range(ord('a'), ord('z') + 1)])

# Define a function to split text into words and remove specified words
def split_into_words_and_remove_stopwords(text, stopwords):
    # Split text using various separators (space, period, comma)
    words = re.split(r'\s+|[,\.:!?"&]', text)
    # Remove specified stopwords from the list
    words = [word.lower() for word in words if word.lower() not in stopwords]
    return words


# Apply the function to the "storyline" column
df['Storyline'] = df['Storyline'].apply(lambda x: split_into_words_and_remove_stopwords(x, stopwords_list))
df['Genre'] = df['Genre'].apply(lambda x: split_into_words_and_remove_stopwords(x, stopwords_list))
df['Stars'] = df['Stars'].apply(lambda x: x.split(','))
df['Writer'] = df['Writer'].apply(lambda x: x.split(','))

test['Stars'] = test['Stars'].apply(lambda x: x.split(','))
test['Genre'] = test['Genre'].apply(lambda x: split_into_words_and_remove_stopwords(x, stopwords_list))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\2132637258.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Storyline'] = df['Storyline'].apply(lambda x: split_into_words_and_remove_stopwords(x, stopwords_list))
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\2132637258.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genre'] = df['

In [6]:
# lấy star[0] trong cột Stars để làm 1 feature riêng, là diễn viên chính
df['Star'] = df['Stars'].apply(lambda x: x[0] if x else None)
test['Star'] = test['Stars'].apply(lambda x: x[0] if x else None)

genres = ['news','western','musical','sport','war','animation','documentary','music','family','history','sci-fi','fantasy','biography','horror','adventure','mystery','action','romance','crime','thriller','comedy','drama']


C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\677463093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Star'] = df['Stars'].apply(lambda x: x[0] if x else None)


In [7]:
#input: Rating, Rating Count, Certificate, Director, Star
#output: trả về luôn cột df['Genre'] là true/false cho phim (tôi update cho cả bộ df luôn)

numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

num = ['Rating', 'Rating Count']
total = ['Rating', 'Rating Count', 'Certificate', 'Director', 'Star']

ans = []

for genre in genres:
    numeric_features = ['Rating', 'Rating Count']
    categorical_features = ['Certificate', 'Director', 'Star']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    X = df[['Certificate', 'Director', 'Star', 'Rating', 'Rating Count']]
    y = [genre in x for x in df['Genre'].values]

    X_preprocessed = preprocessor.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

    model = make_pipeline(
        RandomOverSampler(random_state=42),
        LogisticRegression()
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(genre)

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-score: {f1}')
    
    # update cho cả bộ df
    df[genre] = model.predict(X_preprocessed)
    num.append(genre)
    total.append(genre)
    ans.append([genre, accuracy, recall])


news
Accuracy: 0.9989334470989761
Precision: 0.0
Recall: 0.0
F1-score: 0.0


C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\1599282026.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = model.predict(X_preprocessed)
C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\15992820

western
Accuracy: 0.9929607508532423
Precision: 0.0
Recall: 0.0
F1-score: 0.0


C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\1599282026.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = model.predict(X_preprocessed)


musical
Accuracy: 0.984641638225256
Precision: 0.13333333333333333
Recall: 0.08
F1-score: 0.1


...

C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\1599282026.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = model.predict(X_preprocessed)


thriller
Accuracy: 0.7734641638225256
Precision: 0.375
Recall: 0.3919239904988123
F1-score: 0.38327526132404177


C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\1599282026.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = model.predict(X_preprocessed)


comedy
Accuracy: 0.7367747440273038
Precision: 0.6302395209580839
Recall: 0.5322376738305942
F1-score: 0.5771076079506512
drama
Accuracy: 0.6928327645051194
Precision: 0.7210506844247133
Recall: 0.7396584440227704
F1-score: 0.7302360434619708


C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\1599282026.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = model.predict(X_preprocessed)


In [8]:
ans

[['news', 0.9989334470989761, 0.0],
 ['western', 0.9929607508532423, 0.0],
 ['musical', 0.984641638225256, 0.08],
 ['sport', 0.9633105802047781, 0.10526315789473684],
 ['war', 0.9690699658703071, 0.05405405405405406],
 ['animation', 0.9505119453924915, 0.6065573770491803],
 ['documentary', 0.9549914675767918, 0.5069767441860465],
 ['music', 0.9511518771331058, 0.09917355371900827],
 ['family', 0.9272610921501706, 0.5359116022099447],
 ['history', 0.9332337883959044, 0.1301775147928994],
 ['sci-fi', 0.919155290102389, 0.23076923076923078],
 ['fantasy', 0.9116894197952219, 0.18292682926829268],
 ['biography', 0.9031569965870307, 0.291044776119403],
 ['horror', 0.861561433447099, 0.5308056872037915],
 ['adventure', 0.8675341296928327, 0.5078125],
 ['mystery', 0.8547354948805461, 0.16911764705882354],
 ['action', 0.8171928327645052, 0.5364583333333334],
 ['romance', 0.7860494880546075, 0.3328912466843501],
 ['crime', 0.7805034129692833, 0.34191176470588236],
 ['thriller', 0.773464163822525

In [12]:
numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

numeric_features = num.copy()
categorical_features = ['Certificate', 'Director', 'Star']

genre = 'drama'

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = df[total]
y = [genre in x for x in df['Genre'].values]

X_preprocessed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

model = make_pipeline(
    RandomOverSampler(random_state=42),
    LogisticRegression()
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
    
print(genre)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')
    
df[genre] = model.predict(X_preprocessed)
ans.append([genre, accuracy, recall])


drama
Accuracy: 0.6860068259385665
Precision: 0.7014201593349497
Recall: 0.7685009487666035
F1-score: 0.7334299166968489


C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kingk\AppData\Local\Temp\ipykernel_7456\2571254094.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[genre] = model.predict(X_preprocessed)


In [9]:
for genre in df:
    genre = sorted(genre)

# Sort the genres
genres = sorted(genres)

# Create a new column 'genre_checked' with a list of sorted genres for each row
df['genre_checked'] = df.apply(lambda row: sorted([genre for genre in genres if row[genre]]), axis=1)

df['genre_checked']

C:\Users\kingk\AppData\Local\Temp\ipykernel_6488\1338331831.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre_checked'] = df.apply(lambda row: sorted([genre for genre in genres if row[genre]]), axis=1)


0        [action, adventure, crime, fantasy, horror, my...
1        [action, adventure, fantasy, horror, sci-fi, t...
2                       [biography, crime, drama, history]
3        [action, adventure, fantasy, horror, sci-fi, t...
4           [action, adventure, fantasy, sci-fi, thriller]
                               ...                        
23772                                             [horror]
23774                                      [comedy, drama]
23775                                             [comedy]
23776                                     [drama, romance]
23777                             [crime, drama, thriller]
Name: genre_checked, Length: 23436, dtype: object

In [13]:
df[df['genre_checked'] == df['Genre']]

print("Accuracy:", num_same_rows / len(df))

Accuracy: 0.4453405017921147
